<a href="https://colab.research.google.com/github/vicotrbb/machine_learning/blob/master/data_mining/wikipedia_content_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crawler de conteudo da wikipedia

A ideia deste projeto é criar um crawler capaz de ler e criar um grande dataset com o conteudo do wikipedia, abrangindo todo tipo de informação possivel.

O dataset pode ser destinado para treinar modelos de machine learning de NLP, como o BERT.


In [ ]:
!pip install wikipedia

## Criando o crawler da wikipedia

In [ ]:
import wikipedia
import time
import re
import boto3
import json

In [ ]:
page = wikipedia.page("USA")
print(page.categories)

article = page.content
article = re.sub(r'={1,}.+={1,}', '', article)
article = article.split('\n{1,}')
article.append(page.summary)
print(article)


['1776 establishments in the United States', 'All Wikipedia articles in need of updating', 'All Wikipedia articles written in American English', 'All articles containing potentially dated statements', 'All articles with unsourced statements', 'Articles containing Latin-language text', 'Articles containing potentially dated statements from 2001', 'Articles containing potentially dated statements from 2017', 'Articles containing potentially dated statements from 2018', 'Articles containing potentially dated statements from June 2018', 'Articles containing potentially dated statements from September 2014', 'Articles with short description', 'Articles with unsourced statements from July 2020', 'CS1: long volume value', 'CS1 errors: dates', 'CS1 maint: ref=harv', 'CS1 maint: uses authors parameter', 'Coordinates on Wikidata', 'Countries in North America', 'English-speaking countries and territories', 'Federal constitutional republics', 'Former British colonies', 'Former confederations', 'G2

In [ ]:
rand = wikipedia.random(pages=1)
print(rand)

Getting Closer


In [ ]:
def parsePageContent(article):
    content = re.sub(r'={1,}.+={1,}', '', article)
    content = content.split('\n{1,}')
    return content

In [ ]:
def executeCheckpoint(topics):
    s3 = boto3.client('s3')
    file = 'wikipedia-content-dataset.json'
    bucket = 'wikipedia-crawler-s3'

    with open(file, 'w') as outfile:
        json.dump(topics, outfile)

    s3.upload_file(file, bucket, file)
    os.remove(file)
    return 0

In [ ]:
def setupLogger():
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [ ]:
import wikipedia
import time
import re
import sys
import boto3
import json
import os
import logging

def crawl():
    visited_topics = []
    topics = {}
    content_goal = 25000
    checkpoint_treshold = 25
    checkpoint_counter = 0

    while len(visited_topics) <= content_goal:
        topic = wikipedia.random(pages=1)
        if topic not in visited_topics:
            try:
                page = wikipedia.page(topic)
                if page is not None or page != '':
                    visited_topics.append(topic)
                    logging.info(f'Visiting topic: {topic}')
                else:
                    continue

                article = page.content
                content = parsePageContent(article)
                content.append(page.summary)
                topics[topic] = content

                if checkpoint_counter >= checkpoint_treshold:
                    checkpoint_counter = executeCheckpoint(topics)
                    logging.info(f'Checkpoint reached, {len(visited_topics)} visited')
                else:
                    checkpoint_counter += 1

                time.sleep(1)
            except Exception as ex:
                logging.error(ex)
                continue
        else:
            continue

setupLogger()
crawl()

In [ ]:
topics

{'Acrocanitidae': ['Acrocanitidae is one of seven families of the Prionocerataceae families, a member of the Goniatitida order. They are an extinct group of ammonoid, which are shelled cephalopods related to squids, belemnites, octopuses, and cuttlefish, and more distantly to the nautiloids.\n\n\n== References ==\nNeues Jahrbuch für Geologie und Paläontologie - Abhandlungen, Volume 243, Number 2, February 2007, pp. 127–148(22)\nThe Paleobiology Database accessed on 10/01/07',
  'Acrocanitidae is one of seven families of the Prionocerataceae families, a member of the Goniatitida order. They are an extinct group of ammonoid, which are shelled cephalopods related to squids, belemnites, octopuses, and cuttlefish, and more distantly to the nautiloids.\n\n'],
 'Crossed molecular beam': ['Crossed molecular beam experiments are chemical experiments where two beams of atoms or molecules are collided together to study the dynamics of the chemical reaction, and can detect individual reactive coll